In [1]:
import pandas as pd
import json
from notebookutils import mssparkutils
from pyspark.sql import functions as F
from datetime import datetime

def generateValidationHtml(result, max_unexpected=5):
    """
    Generate an HTML report from a Great Expectations CheckpointResult object.
    """
    # Access attributes directly
    run_id = getattr(result, "run_id", {})

    # Grab runtime and nicely format it
    run_time_dt = getattr(run_id,"run_time")
    run_time = run_time_dt.strftime("%Y-%m-%d %H:%M:%S %Z")
    
    # Extract first validation result
    run_results = getattr(result, "run_results", {})
    first_key = next(iter(run_results.keys()))
    validation_json = run_results[first_key]


    # Collect rows for HTML table
    rows = []
    for res in validation_json.get("results", []):
        cfg = res.get("expectation_config", {})
        res_result = res.get("result", {})

        rows.append({
            "Title": cfg.get("meta", {}).get("title", ""),
            "Description": cfg.get("meta", {}).get("description", ""),
            "Column": cfg.get("kwargs", {}).get("column", ""),
            "Expectation": cfg.get("type", ""),
            "Success": res.get("success", False),
            "Unexpected Count": res_result.get("unexpected_count", 0),
            "Unexpected %": round(res_result.get("unexpected_percent", 0), 2),
            "Unexpected Values (sample)": ", ".join(
                map(str, res_result.get("unexpected_list", [])[:max_unexpected])
            )
        })
    
    

    df = pd.DataFrame(rows)

    html_output = getHTML(df, run_time)

    return html_output

StatementMeta(, , -1, SessionStarting, , SessionStarting)

In [ ]:
def getHTML(df, run_time, title="Validation Report"):

    failed_count = df.loc[df["Success"] == False, "Unexpected Count"].sum()

    styled_html = df.to_html(index=False, escape=False, border=0, classes="table table-bordered table-striped")

    # Build final HTML output
    html_output = f"""
    
    <html>
        <head>
            <style>
            body {{
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                margin: 5px;
                background-color: #f9fafb;
                color: #333;
            }}
            h2 {{
                text-align: left;
                color: #2c3e50;
            }}
            p {{
                text-align: left;
                font-size: 14px;
                color: #555;
                font-weight: bold;
            }}
            table {{
                margin: 20px auto;
                width: 100%;
                background: white;
                box-shadow: 0 2px 8px rgba(0,0,0,0.1);
                border-radius: 8px;
            }}
            thead {{
                background: #2c3e50;
                color: white;
                text-transform: uppercase;
                letter-spacing: 0.03em;
                font-size: 13px;
            }}
            th, td {{
                padding: 12px 15px;
                border-bottom: 1px solid #e0e0e0;
                text-align: left;
            }}
            tr:nth-child(even) {{
                background-color: #fafafa;
            }}
            .fail {{
                background-color: #fde2e2 !important;
                color: #b71c1c;
                font-weight: bold;
            }}
            .success {{
                background-color: #e6f4ea !important;
                color: #1b5e20;
                font-weight: bold;
            }}
            .alert {{
                color: red;
                font-weight: bold;
            }}
            </style>
        </head>
        <body>
            <h2>{title}</h2>
            <p>Run Time: {run_time}</p>
            <p class ="alert">{failed_count} record(s) dropped due to validation error. Please review.</p>
            {styled_html}
        </body>
    </html>
    """
    return html_output


In [ ]:
# Get cleaned data and failed records
def cleanData(df, checkpoint_result):
    run_results = getattr(checkpoint_result, "run_results", {})
    failed_queries = []

    for _, validation in run_results.items():
        for res in validation.get("results", []):
            if not res.get("success", True):
                raw_query = res.get("result", {}).get("unexpected_index_query")
                if raw_query:
                    # Remove Python wrapper if present
                    if "F.expr(" in raw_query:
                        raw_query = raw_query.split("F.expr(")[-1].rstrip(")").strip()

                    # Ensure parentheses balance
                    open_parens = raw_query.count("(")
                    close_parens = raw_query.count(")")
                    if open_parens > close_parens:
                        raw_query += ")" * (open_parens - close_parens)

                    failed_queries.append(f"({raw_query})")

    if failed_queries:
        combined = " OR ".join(failed_queries)
        print("Final combined SQL:\n", combined)  # debug

        failed_df = df.filter(F.expr(combined))
        cleaned_df = df.exceptAll(failed_df)
        return cleaned_df, failed_df

    return df, None


In [ ]:
# Summarize report
def summarizeReport(result):
    html_report = generateValidationHtml(result)
    run_results = getattr(result, "run_results", {})

    # take overall success (from first validation block, usually sufficient)
    first_key = next(iter(run_results.keys()))
    validation_json = run_results[first_key]

    # overall success
    overall_success = validation_json.get("success", False)

    # count failed rows across all expectations
    total_failed = sum(
        res.get("result", {}).get("unexpected_count", 0)
        for res in validation_json.get("results", [])
    )

    # create a details/title string
    summary = f"Validation Failure - {total_failed} row(s) failed checks"

    output = {
        "success": overall_success,
        "summary": summary,   # 🔹 new field for your title
        "details": html_report
    }
    return json.dumps(output, indent=2)


In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import json
from notebookutils import mssparkutils

# Send Validation alert
def sendNotification(report):

    workspace = "Logistics"
    lakehouse = "Shipments_LH"

    # Path to email/app credentials
    path = f"abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{lakehouse}.Lakehouse/Files/notification/notify.json"

    json_text = mssparkutils.fs.head(path)

    cred = json.loads(json_text)

    # SMTP settings
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    username = cred["username"]
    password = cred["password"]  # or app password if MFA

    # Create the email
    msg = MIMEMultipart("alternative")
    msg["Subject"] = report["summary"] # Grab summary from  report json
  
    msg["From"] = username
    msg["To"] = cred["target_email"]

    html_part = MIMEText(report["details"], "html") # Grab  html report and table 
    msg.attach(html_part)

    with smtplib.SMTP(smtp_server, smtp_port) as server:             
        server.starttls()            
        server.login(username, password)
        server.send_message(msg)

# Poke around built docs    
# !cp gx/uncommitted/data_docs/local_site/index.html /lakehouse/default/Files 


In [ ]:
# Write file to Lakehouse
def writetToLH(data, path):
    mssparkutils.fs.put(path, data, overwrite=True)
    print(f"Saved HTML report to Lakehouse at: {path}")